In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
training_data = datasets.FashionMNIST(
    train=True,
    transform=ToTensor(),
    download=True,
    root='data'
)

test_data = datasets.FashionMNIST(
    root='data',
    download=True,
    transform=ToTensor(),
    train=False
)

100%|██████████| 26421880/26421880 [00:01<00:00, 18249084.34it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 287909.81it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5550960.29it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 11703131.16it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
device = (
    'cuda' if torch.cuda.is_available() else 'cpu'
)

print(f"Using {device} device")

Using cpu device


In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()

  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
  print(f"Epoch: {t+1}\n------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done")

Epoch: 1
------------------------
loss: 2.307316 [   64/60000]
loss: 2.294809 [ 6464/60000]
loss: 2.268579 [12864/60000]
loss: 2.262986 [19264/60000]
loss: 2.257206 [25664/60000]
loss: 2.228195 [32064/60000]
loss: 2.237793 [38464/60000]
loss: 2.206419 [44864/60000]
loss: 2.191752 [51264/60000]
loss: 2.166111 [57664/60000]
Test Error: 
 Accuracy: 44.5%, Avg loss: 2.164060 

Epoch: 2
------------------------
loss: 2.181288 [   64/60000]
loss: 2.171366 [ 6464/60000]
loss: 2.108002 [12864/60000]
loss: 2.118260 [19264/60000]
loss: 2.084996 [25664/60000]
loss: 2.022558 [32064/60000]
loss: 2.052957 [38464/60000]
loss: 1.976232 [44864/60000]
loss: 1.966843 [51264/60000]
loss: 1.902168 [57664/60000]
Test Error: 
 Accuracy: 54.2%, Avg loss: 1.904265 

Epoch: 3
------------------------
loss: 1.943655 [   64/60000]
loss: 1.918300 [ 6464/60000]
loss: 1.795068 [12864/60000]
loss: 1.827301 [19264/60000]
loss: 1.740190 [25664/60000]
loss: 1.678415 [32064/60000]
loss: 1.703632 [38464/60000]
loss: 1.601

# Tensors

In [ ]:
import torch
import numpy as np

In [ ]:
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)

In [ ]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

In [ ]:
x_ones = torch.ones_like(x_data)
print(f"Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float)
print(f"Tensor: \n {x_rand} \n")

Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Tensor: 
 tensor([[0.5923, 0.0012],
        [0.4990, 0.2615]]) 



In [ ]:
shape = (5, 2, 4)

tensor_one = torch.ones(shape)
tensor_rand = torch.rand(shape)
tensor_zero = torch.zeros(shape)

print(f"Tensor: \n {tensor_one} \n")
print(f"Tensor: \n {tensor_rand} \n")
print(f"Tensor: \n {tensor_zero} \n")

Tensor: 
 tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.]]]) 

Tensor: 
 tensor([[[0.7677, 0.5900, 0.5250, 0.8857],
         [0.7880, 0.3378, 0.8123, 0.5675]],

        [[0.5431, 0.6184, 0.7395, 0.0870],
         [0.1053, 0.1169, 0.9323, 0.4232]],

        [[0.5179, 0.7287, 0.5106, 0.7562],
         [0.7919, 0.4161, 0.5821, 0.3447]],

        [[0.0054, 0.4446, 0.2166, 0.5242],
         [0.9331, 0.8972, 0.2350, 0.3296]],

        [[0.3759, 0.8352, 0.5668, 0.4259],
         [0.4062, 0.3368, 0.9110, 0.6016]]]) 

Tensor: 
 tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 

In [ ]:
tensor = torch.rand(3, 4)

print(f"Tensor's datatype: {tensor.dtype}")
print(f"Tensor's shape: {tensor.shape}")
print(f"Tensor's device: {tensor.device}")

Tensor's datatype: torch.float32
Tensor's shape: torch.Size([3, 4])
Tensor's device: cpu


In [ ]:
if torch.cuda.is_available():
  tensor = tensor.to('cuda')

In [ ]:
tensor = torch.ones(4, 4)
print(f"The first row: {tensor[0]}")
print(f"The second column: {tensor[:, 1]}")
print(f"The last column: {tensor[..., -1]}")
tensor[:, 2] = 69
print(tensor)

The first row: tensor([1., 1., 1., 1.])
The second column: tensor([1., 1., 1., 1.])
The last column: tensor([1., 1., 1., 1.])
tensor([[ 1.,  1., 69.,  1.],
        [ 1.,  1., 69.,  1.],
        [ 1.,  1., 69.,  1.],
        [ 1.,  1., 69.,  1.]])


In [ ]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[ 1.,  1., 69.,  1.,  1.,  1., 69.,  1.,  1.,  1., 69.,  1.],
        [ 1.,  1., 69.,  1.,  1.,  1., 69.,  1.,  1.,  1., 69.,  1.],
        [ 1.,  1., 69.,  1.,  1.,  1., 69.,  1.,  1.,  1., 69.,  1.],
        [ 1.,  1., 69.,  1.,  1.,  1., 69.,  1.,  1.,  1., 69.,  1.]])


In [ ]:
y1 = tensor.mm(tensor.T)
y2 = torch.rand_like(y1)
res = torch.matmul(tensor, tensor.T, out=y2)
print(res)

tensor([[4764., 4764., 4764., 4764.],
        [4764., 4764., 4764., 4764.],
        [4764., 4764., 4764., 4764.],
        [4764., 4764., 4764., 4764.]])


In [ ]:
agg = tensor.sum()
agg_item = agg.item()
print(f"The sum: {agg_item} \nThe datatype: {type(agg_item)}")

The sum: 288.0 
The datatype: <class 'float'>


In [ ]:
print(tensor)
tensor.add_(3)
print(tensor)

tensor([[ 1.,  1., 69.,  1.],
        [ 1.,  1., 69.,  1.],
        [ 1.,  1., 69.,  1.],
        [ 1.,  1., 69.,  1.]])
tensor([[ 4.,  4., 72.,  4.],
        [ 4.,  4., 72.,  4.],
        [ 4.,  4., 72.,  4.],
        [ 4.,  4., 72.,  4.]])
